#### source:
https://www.geeksforgeeks.org/python-removing-unwanted-characters-from-string/

# Extract data from gamePass

In [1]:
#Extracty data from gamePass
import os
os.system('node Game-Pass-API/index.js')

1

In [2]:
import pandas as pd
import numpy as np
gamepass_games=pd.read_json("Game-Pass-API/output/formattedGameProperties_console_US.json")
gamepass_games

,productTitle
0,7 Days to Die
1,A Little to the Left
2,A Plague Tale: Requiem
3,A Short Hike
4,A Way Out
...,...
497,You Suck at Parking® Complete Edition
498,Zombie Army 4: Dead War
499,Zoo Tycoon: Ultimate Animal Collection
500,Zuma


# transformation: need to lowercase and remove special characters

In [3]:
gamepass_games["productTitle"][25]="Banjo-Kazooie: Nuts & Bolts"

In [4]:
gamepass_games["productTitle"][25]

'Banjo-Kazooie: Nuts & Bolts'

In [5]:
import unidecode

list_transformed_title=[]
# initializing bad_chars_list
# r copyright -> spe@#$ci87al*& ou ®
bad_chars = [';', ':', '!','\'', "*", "®","™"]
# using replace() to
# remove bad_chars

for title in gamepass_games['productTitle']:
    title = ''.join((filter(lambda i: i not in bad_chars, 
                              title))).replace(" Console","").replace(" Xbox Series X|S","").replace(" Xbox One","").replace("&","and").replace(" ","-").lower()
    list_transformed_title.append(unidecode.unidecode(title))
    

In [6]:
gamepass_games["tr_productTitle"]=pd.DataFrame(list_transformed_title)

In [7]:
gamepass_games["productTitle"][51]

'Brütal Legend'

# Extract data from Metacritic

In [8]:
from bs4 import BeautifulSoup
import requests, random


AGENTS = ('Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
          'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36')

#create headers
HEADERS               = requests.utils.default_headers()
HEADERS['User-Agent'] = random.choice(AGENTS)

In [9]:
"""
url = "https://www.metacritic.com/game/cities-skylines---remastered/"
page = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(page.content, 'html.parser')
score_text = soup.find("span", {"class": "c-productScoreInfo_scoreSentiment"})
if score_text is None:
    print("not good")
"""

'\nurl = "https://www.metacritic.com/game/cities-skylines---remastered/"\npage = requests.get(url, headers=HEADERS)\nsoup = BeautifulSoup(page.content, \'html.parser\')\nscore_text = soup.find("span", {"class": "c-productScoreInfo_scoreSentiment"})\nif score_text is None:\n    print("not good")\n'

In [10]:
list_Of_Score=[]
it=0
for tr_title in gamepass_games["tr_productTitle"]:
    it+=1
    url = "https://www.metacritic.com/game/"+tr_title+"/"
    #print(url,it-1)
    page = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(page.content, 'html.parser')
    score_text = soup.find("span", {"class": "c-productScoreInfo_scoreSentiment"})
    if score_text is None:
        #print("not good")
        list_Of_Score.append("not good")
    else:
        list_Of_Score.append(score_text.get_text().replace("\n ","").replace("\n",""))
gamepass_games["metascore"]=pd.DataFrame(list_Of_Score)

In [11]:
gamepass_games

,productTitle,tr_productTitle,metascore,score
0,7 Days to Die,7-days-to-die,Generally Unfavorable,Generally Unfavorable
1,A Little to the Left,a-little-to-the-left,Generally Favorable,Generally Favorable
2,A Plague Tale: Requiem,a-plague-tale-requiem,Generally Favorable,Generally Favorable
3,A Short Hike,a-short-hike,Generally Favorable,Generally Favorable
4,A Way Out,a-way-out,Generally Favorable,Generally Favorable
...,...,...,...,...
497,You Suck at Parking® Complete Edition,you-suck-at-parking-complete-edition,not good,not good
498,Zombie Army 4: Dead War,zombie-army-4-dead-war,Mixed or Average,Mixed or Average
499,Zoo Tycoon: Ultimate Animal Collection,zoo-tycoon-ultimate-animal-collection,Mixed or Average,Mixed or Average
500,Zuma,zuma,Available after 4 ratings,Available after 4 ratings


In [13]:
gamepass_games["metascore"][gamepass_games["metascore"]!="not good"].count()

421

In [14]:
gamepass_games["metascore"][gamepass_games["metascore"]=="not good"].count()

81